# Articulations in Newton

[![ Click here to deploy.](https://brev-assets.s3.us-west-1.amazonaws.com/nv-lb-dark.svg)](https://brev.nvidia.com/launchable/deploy?launchableID=env-3581zHTIAb8sXpu1DzaQv73vKqS)

**Articulations** are multi-body systems connected by joints, such as robotic arms, humanoids, or quadrupeds. This tutorial demonstrates how to create articulated systems in Newton using the `ModelBuilder` API.

In this notebook, you'll learn:
- Creating articulations with different joint types
- Building articulated systems programmatically
- Importing articulations from URDF, MJCF, and USD files
- Configuring joint properties and initial states

## What are Articulations?

An articulation is a collection of rigid bodies (links) connected by joints. Each joint constrains the relative motion between two bodies:

- **REVOLUTE (Hinge)**: Rotation around a single axis
- **PRISMATIC (Slider)**: Translation along a single axis
- **BALL (Spherical)**: Rotation in all directions
- **FIXED**: No relative motion (welded connection)
- **DISTANCE**: Maintains constant distance between points

Newton supports both **maximal coordinate** (each body has 6 DOF) and **reduced coordinate** (generalized coordinates) representations.


## Setup and Imports


In [ ]:
import newton
import warp as wp
import numpy as np
from pxr import Usd, UsdGeom
import newton.examples
import newton.usd
import rerun as rr
from tqdm.notebook import trange


## Double Pendulum

Let's create a double pendulum with two links connected by revolute joints.

Each joint in Newton has a parent and child body as well as a parent transform `x_pj` and a child transform `x_cj`.


Depending on the joint type, a joint has different degrees of freedom (dof). For example, a revolute joint has a single degree of freedom that defines the angle of rotation (in radians) about a single joint axis.
Joints, such as D6 joints in Newton, can have up to 6 degrees of freedom.

These degrees of freedom are called *generalized coordinates*. In Newton, we store them in the `joint_q` array (joint positions) and the joint velocities in generalized coordinates in `joint_qd`.
The `ModelBuilder` and the `Model` both have definitions for `joint_q` and `joint_qd`. Those are used to initialize the `State.joint_q` and `State.joint_qd` arrays that are updated during the simulation.

Given these joint dofs, the joint transform `x_j` is computed based on the type of joint and the corresponding joint axes. For a revolute joint `x_j` will always have zero translation and only a rotation about the joint axis:
```py
x_j = wp.transform(wp.vec3(0.0, 0.0, 0.0), wp.quat_from_axis_angle(axis, q)).
```

If we want to compute the body transforms `body_q` and velocities `body_qd` in world space (maximal coordinates) of an articulated system given its generalized coordinates, we have to evaluate the system's forward kinematics. This function traverses the kinematic tree defined by the articulation to compute the maximal coordinates of each child body `x_wc` given the maximal coordinates of its parent `x_wp`:

<p align="center"><img src="https://newton-physics.github.io/newton/_images/joint_transforms.png" alt="Joint transforms" width="400" /></p>


$$
x\_wc = x\_wp \cdot x\_pj \cdot x\_j \cdot x\_cj^{-1}
$$

See https://newton-physics.github.io/newton/concepts/articulations.html for more details.

In [ ]:
# Create a new builder for the double pendulum
builder = newton.ModelBuilder()
builder.add_articulation(key="double_pendulum")

# Link dimensions
hx = 1.0
hy = 0.1
hz = 0.1

# Create first link
link_0 = builder.add_body(key="link_0")
builder.add_shape_box(link_0, hx=hx, hy=hy, hz=hz)

# Create second link
link_1 = builder.add_body(key="link_1")
builder.add_shape_box(link_1, hx=hx, hy=hy, hz=hz)

# Joint 1: World to first link
builder.add_joint_revolute(
    parent=-1,  # -1 means world
    child=link_0,
    axis=wp.vec3(0.0, 1.0, 0.0),
    parent_xform=wp.transform(p=wp.vec3(0.0, 0.0, 5.0), q=wp.quat_identity()),
    child_xform=wp.transform(p=wp.vec3(-hx, 0.0, 0.0), q=wp.quat_identity()),
    key="joint_0"
)

# Joint 2: First link to second link
builder.add_joint_revolute(
    parent=link_0,
    child=link_1,
    axis=wp.vec3(0.0, 1.0, 0.0),
    parent_xform=wp.transform(p=wp.vec3(hx, 0.0, 0.0), q=wp.quat_identity()),
    child_xform=wp.transform(p=wp.vec3(-hx, 0.0, 0.0), q=wp.quat_identity()),
    key="joint_1"
)

print(f"Double pendulum created with {builder.joint_count} joints")


Now we define the initial joint angles (generalized coordinates). See https://newton-physics.github.io/newton/concepts/articulations.html for more details.

In [ ]:
builder.joint_q[0] = 0.1
builder.joint_q[1] = -0.4

Let's create the model and visualize the result.

We will see that it is necessary to evaluate the forward kinematics via `newton.eval_fk()` which converts from generalized coordinates (`joint_q`, `joint_qd`) to maximal coordinates (`body_q`, `body_qd`).

Maximal coordinates are used by solvers, such as PhysX (not part of Newton), SolverXPBD, and SolverSemiImplicit.

Solvers, such as MuJoCo and Featherstone, are based on generalized coordinates.

For contact evaluation and to visualize the state of the system, we need to compute the maximal coordinates.

The generalized-coordinate solvers MuJoCo and Featherstone both update the maximal coordinates during the `Solver.step()` function call.


In [ ]:
# Finalize and simulate
model = builder.finalize()
state = model.state()

newton.eval_fk(model, model.joint_q, model.joint_qd, state)


print(f"Model finalized: {model.body_count} bodies, {model.joint_count} joints")

viewer = newton.viewer.ViewerRerun(keep_historical_data=True)
viewer.set_model(model)
viewer.log_state(state)
viewer


We can modify the joint coordinates on the fly. Note that this is purely kinematic behavior, we have not created any solver yet to evaluate the dynamics of the double pendulum.

In [ ]:
viewer = newton.viewer.ViewerRerun(keep_historical_data=True)
viewer.set_model(model)

for i in trange(500):
    viewer.begin_frame(i / 30)
    # State.joint_q is Warp array (with dimension of all the joint dofs, see `Model.joint_dof_count`)
    state.joint_q.assign([i * 0.1, i * 0.2])
    newton.eval_fk(model, state.joint_q, state.joint_qd, state)
    viewer.log_state(state)
    viewer.end_frame()
viewer

We will now use [MuJoCo Warp](https://github.com/google-deepmind/mujoco_warp) to simulate this double pendulum.

MuJoCo and MuJoCo Warp are available in Newton through the `SolverMuJoCo` class.

In [ ]:
# Simulation parameters
fps = 60                           # Frames per second for visualization
frame_dt = 1.0 / fps        # Time step per frame
sim_time = 0.0              # Current simulation time
sim_substeps = 10                  # Number of physics substeps per frame
sim_dt = frame_dt / sim_substeps   # Physics time step

print(f"Simulation configured:")
print(f"  Frame rate: {fps} Hz")
print(f"  Frame dt: {frame_dt:.4f} s")
print(f"  Physics substeps: {sim_substeps}")
print(f"  Physics dt: {sim_dt:.4f} s")


We create a helper method to simulate and visualize the behavior more easily.

In [ ]:
def simulate(model: newton.Model, solver: newton.solvers.SolverBase, sim_substeps=10):
    state_0 = model.state()
    state_1 = model.state()
    control = model.control()
    contacts = model.collide(state_0)
    
    def advance():
        """Run multiple physics substeps for one frame."""
        nonlocal state_0, state_1, contacts
        
        for _ in range(sim_substeps):
            # Clear accumulated forces
            state_0.clear_forces()
            
            # Advance the simulation by one physics timestep
            solver.step(state_0, state_1, control, contacts, sim_dt)
            
            # Swap states (next becomes current)
            state_0, state_1 = state_1, state_0

    # Capture the simulation as a CUDA graph (if running on GPU)
    if wp.get_device().is_cuda:
        with wp.ScopedCapture() as capture:
            advance()
        graph = capture.graph
        print("CUDA graph captured for optimized execution")
    else:
        graph = None
        print("Running on CPU (no CUDA graph)")

    # Create the Rerun viewer
    viewer = newton.viewer.ViewerRerun(keep_historical_data=True)

    # Set the model (this logs the static geometry)
    viewer.set_model(model)

    # Run the simulation
    num_frames = 500

    sim_time = 0.0

    for frame in trange(num_frames, desc="Simulating"):
        # Execute the simulation (use CUDA graph if available)
        if graph:
            wp.capture_launch(graph)
        else:
            simulate()
        
        # Log the current state to the viewer
        viewer.begin_frame(sim_time)
        viewer.log_state(state_0)
        viewer.log_contacts(contacts, state_0)
        viewer.end_frame()
        
        # Advance simulation time
        sim_time += frame_dt

    print(f"\nSimulation complete! Total time: {sim_time:.2f} seconds")
    return viewer

In [ ]:
solver = newton.solvers.SolverMuJoCo(model)
simulate(model, solver)

## Importing assets

Newton can import articulated systems from USD, URDF, and MJCF files.

We start by loading a Franka robot arm.


In [ ]:
quadruped = newton.ModelBuilder()

# set default parameters for the quadruped
quadruped.default_body_armature = 0.01
quadruped.default_joint_cfg.armature = 0.01
quadruped.default_joint_cfg.target_ke = 2000.0
quadruped.default_joint_cfg.target_kd = 1.0
quadruped.default_shape_cfg.ke = 1.0e4
quadruped.default_shape_cfg.kd = 1.0e2
quadruped.default_shape_cfg.kf = 1.0e2
quadruped.default_shape_cfg.mu = 1.0

# parse the URDF file
quadruped.add_urdf(
    newton.examples.get_asset("quadruped.urdf"),
    xform=wp.transform(wp.vec3(0.0, 0.0, 0.7), wp.quat_identity()),
    floating=True,
    enable_self_collisions=False,
)

# set initial joint positions
quadruped.joint_q[-12:] = [0.2, 0.4, -0.6, -0.2, -0.4, 0.6, -0.2, 0.4, -0.6, 0.2, -0.4, 0.6]
quadruped.joint_target_pos[-12:] = quadruped.joint_q[-12:]

In [ ]:
# use "scene" for the entire set of worlds
scene = newton.ModelBuilder()

# use the builder.replicate() function to create N copies of the world
num_worlds = 16
scene.replicate(quadruped, num_worlds)
scene.add_ground_plane()

# finalize model
model = scene.finalize()
solver = newton.solvers.SolverMuJoCo(model, njmax=50)

In [ ]:
simulate(model, solver)

## Importing from MJCF (MuJoCo XML)

Newton also supports **MJCF (MuJoCo XML Format)**, which is widely used in machine learning and reinforcement learning research.

### Loading a Humanoid from MJCF


In [ ]:
# Create builder for MJCF import
builder_mjcf = newton.ModelBuilder()

# Import MJCF file
builder_mjcf.add_mjcf(
    newton.examples.get_asset("nv_humanoid.xml"),
    ignore_names=["floor", "ground"],
    xform=wp.transform(wp.vec3(0, 0, 1.3)),
)

builder_mjcf.add_ground_plane()

print(f"MJCF imported: {builder_mjcf.body_count} bodies, {builder_mjcf.joint_count} joints")

# Finalize and prepare for simulation
model_mjcf = builder_mjcf.finalize()

# We have to apply som
solver_mjcf = newton.solvers.SolverMuJoCo(model_mjcf, iterations=10, njmax=100, nconmax=50)

simulate(model_mjcf, solver_mjcf)


## Importing from USD (Universal Scene Description)

Newton can import articulations from **USD** files, which are commonly used in graphics and simulation pipelines (e.g., NVIDIA Omniverse, Isaac Sim).

### Loading Articulation from USD


In [ ]:

articulation_builder = newton.ModelBuilder(up_axis=newton.Axis.Z)
newton.solvers.SolverMuJoCo.register_custom_attributes(articulation_builder)
articulation_builder.default_joint_cfg = newton.ModelBuilder.JointDofConfig(
    limit_ke=1.0e3, limit_kd=1.0e1, friction=1e-5
)
articulation_builder.default_shape_cfg.ke = 2.0e3
articulation_builder.default_shape_cfg.kd = 1.0e2
articulation_builder.default_shape_cfg.kf = 1.0e3
articulation_builder.default_shape_cfg.mu = 0.75

asset_path = newton.utils.download_asset("anybotics_anymal_d")
asset_file = str(asset_path / "usd" / "anymal_d.usda")
articulation_builder.add_usd(
    asset_file,
    collapse_fixed_joints=False,
    enable_self_collisions=False,
    hide_collision_shapes=True,
)

articulation_builder.joint_q[:3] = [0.0, 0.0, 0.62]
if len(articulation_builder.joint_q) > 6:
    articulation_builder.joint_q[3:7] = [0.0, 0.0, 0.0, 1.0]

for i in range(articulation_builder.joint_dof_count):
    articulation_builder.joint_target_ke[i] = 150
    articulation_builder.joint_target_kd[i] = 5

builder = newton.ModelBuilder(up_axis=newton.Axis.Z)
num_worlds = 4
for _ in range(num_worlds):
    builder.add_builder(articulation_builder)

builder.default_shape_cfg.ke = 1.0e3
builder.default_shape_cfg.kd = 1.0e2
builder.add_ground_plane()

model = builder.finalize()
solver = newton.solvers.SolverMuJoCo(
    model,
    cone="elliptic",
    impratio=100,
    iterations=100,
    ls_iterations=50,
    nconmax=250,
    njmax=100,
)

In [ ]:
simulate(model, solver)

## Key Concepts Summary

### Joint Types

| Joint Type | DOF | Description | Common Use Cases |
|------------|-----|-------------|------------------|
| **REVOLUTE** | 1 | Rotation around axis | Elbows, knees, wheels |
| **PRISMATIC** | 1 | Translation along axis | Sliders, telescoping arms |
| **BALL** | 3 | Free rotation | Shoulders, hips |
| **FIXED** | 0 | Rigid connection | Welded parts |
| **FREE** | 6 | Free translation and rotation | Floating-base articulations (only required for generalized coordinate solvers) |
| **D6** | 1 - 6 | Generic joint that can represent all other joints | Asset importers |

### Creating Articulations

1. **Programmatic Creation**:
   - Use `add_body()` to create links
   - Use `add_joint_*()` to connect them
   - Set initial joint states with `builder.joint_q`

2. **Import from Files**:
   - **URDF**: `builder.add_urdf(path, ...)`
   - **MJCF**: `builder.add_mjcf(path, ...)`
   - **USD**: `builder.add_usd(path, ...)`

### Important Parameters

- **`floating=True`**: Creates a free-floating base (6 DOF)
- **`enable_self_collisions`**: Enable/disable collisions between robot links
- **`armature`**: Numerical stability parameter (adds inertia)
- **Joint limits**: `limit_lower`, `limit_upper` for prismatic/revolute joints
- **Joint modes**: `FORCE`, `TARGET_POSITION`, `TARGET_VELOCITY`

### Forward Kinematics

After setting joint positions, always call:
```python
newton.eval_fk(model, model.joint_q, model.joint_qd, state)
```
This updates body positions based on joint configurations.


## Next Steps

Now that you understand articulations, you can:

1. **Add Actuators**: Control joints with torques or position targets
2. **Implement Controllers**: PD controllers, inverse kinematics, etc.
3. **Train Policies**: Use differentiable simulation for RL
4. **Complex Robots**: Import and simulate humanoids, manipulators, etc.
5. **Multi-World Simulation**: Use `builder.replicate()` for parallel environments

### Useful Resources

- **Newton Examples**: `newton/examples/` directory
- **URDF Tutorial**: http://wiki.ros.org/urdf/Tutorials
- **MJCF Documentation**: https://mujoco.readthedocs.io/en/stable/XMLreference.html
- **USD Documentation**: https://graphics.pixar.com/usd/docs/index.html
